In [1]:
import scipy
import numpy as np
import numpy.linalg as LA

%matplotlib inline
import pylab
import matplotlib.pyplot as plt
from sklearn import neighbors
import pdb


# 数据加载

In [2]:
import scipy.io as sio

def load_Yale_data(randomsplit):
    #matlab文件名
    matfn=u'face_data/Yale/Yale_32x32.mat'
    data=sio.loadmat(matfn)
    face = data['fea']
    label = data['gnd'] - 1 ### python 从0 开始
    face = face  / 255.0

    #### 显示第一张图像
#     f1 = face[5].reshape((32,32)).T
#     pylab.imshow(f1, cmap=plt.cm.gray), pylab.show()

    ###  p = 6, randomsplit = 1
    matfn = u'face_data/Yale/6Train/'+str(randomsplit)+'.mat'
    index = sio.loadmat(matfn)
    testIdx = index['testIdx'] -1    ### python 从0 开始
    trainIdx = index['trainIdx'] -1  ### python 从0 开始

    train_face = face[trainIdx]
    test_face = face[testIdx]
    train_face = np.squeeze(train_face)  ## 将维度为1去掉
    test_face = np.squeeze(test_face)    ## 将维度为1去掉
    train_face = train_face.T ### 将数据维度变为: 样本维度 x 样本个数
    test_face = test_face.T ### 将数据维度变为: 样本维度 x 样本个数
    train_label = np.squeeze(label[trainIdx])
    test_label = np.squeeze(label[testIdx])
    return train_face, test_face, train_label, test_label

def load_ORL_data(randomsplit):
    #matlab文件名
    matfn=u'face_data/ORL/ORL_32x32.mat'
    data=sio.loadmat(matfn)
    face = data['fea']
    label = data['gnd'] - 1 ### python 从0 开始
    face = face  / 255.0

    #### 显示第一张图像
#     f1 = face[5].reshape((32,32)).T
#     pylab.imshow(f1, cmap=plt.cm.gray), pylab.show()

    ###  p = 6, randomsplit = 1
    matfn = u'face_data/ORL/6Train/'+str(randomsplit)+'.mat'
    index = sio.loadmat(matfn)
    testIdx = index['testIdx'] -1    ### python 从0 开始
    trainIdx = index['trainIdx'] -1  ### python 从0 开始

    train_face = face[trainIdx]
    test_face = face[testIdx]
    train_face = np.squeeze(train_face)  ## 将维度为1去掉
    test_face = np.squeeze(test_face)    ## 将维度为1去掉
    train_face = train_face.T ### 将数据维度变为: 样本维度 x 样本个数
    test_face = test_face.T ### 将数据维度变为: 样本维度 x 样本个数
    train_label = np.squeeze(label[trainIdx])
    test_label = np.squeeze(label[testIdx])
    return train_face, test_face, train_label, test_label

# LDE

In [3]:
def EuclideanDistances(A, B):
    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A,BT)
    # print(vecProd)
    SqA =  A**2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B**2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))    
    SqED = sumSqBEx + sumSqAEx - 2*vecProd
    SqED[SqED<0]=0.0   
    ED = np.array(np.sqrt(SqED))
    return ED               

In [4]:
def get_neighbors_ind_and_distance(X, label, K=3, K_bar=3):
    """
    Param:
        X: M * N matrix
        label: the label
        K: The number of neighbors of each point for G  ##  构建同类邻接图 G 的近邻数
        K_bar: The number of neighbors of each point for G_bar  ##  构建同类邻接图 G 的近邻数
    """    
    M, N = X.shape
    distance = EuclideanDistances(X.T, X.T)
    index = np.argsort(distance, axis=0)
    neighbor_ind = []
    neighbor_bar_ind = []
    for i in xrange(len(label)):
        index_i = index[:,i]  ### 第 i 个样本的全部邻居
        neighbor_ind_i = []    #### 与第i个样本是同类样本且是邻居
        neighbor_bar_ind_i = []  #### 与第i个样本不是同类样本且是邻居
        for ind in index_i[1:]: #### 从1开始是因为邻居不包括自己
            if (len(neighbor_ind_i) < K) and (label[i] == label[ind]):
                neighbor_ind_i.append(ind)
            if (len(neighbor_bar_ind_i) < K_bar) and (label[i] != label[ind]):
                neighbor_bar_ind_i.append(ind)
            if (len(neighbor_ind_i) == K) and (len(neighbor_bar_ind_i) == K_bar):
                neighbor_ind.append(neighbor_ind_i)
                neighbor_bar_ind.append(neighbor_bar_ind_i)
                break
    return neighbor_ind, neighbor_bar_ind, distance

In [5]:
def get_weight(neighbor_ind, neighbor_bar_ind, distance, t=5.0):
    N = distance.shape[0]
    Weight = np.zeros((N,N))
    Weight_bar = np.zeros((N,N))
    for i in xrange(N):
        d = distance[i][neighbor_ind[i]]
        Weight[i][neighbor_ind[i]]= np.exp(-d**2/(t))
        d_bar = distance[i][neighbor_bar_ind[i]]
        Weight_bar[i][neighbor_bar_ind[i]]= np.exp(-d_bar**2/t)
    return Weight, Weight_bar  

In [6]:
def get_Laplacian_matrix(W, W_bar):
    D = np.sum(W, axis=1) ### 按行相加
    D = np.diag(D)
    L = D - W 
    D_bar = np.sum(W_bar, axis=1) ### 按行相加
    D_bar = np.diag(D_bar)
    L_bar = D_bar - W_bar 
    return L, L_bar

In [7]:
def get_projections2(X, L, L_bar, d=7):
    A = np.matmul(np.matmul(X, L_bar), X.T)
    B = np.matmul(np.matmul(X, L), X.T)
    eigenvalue, eigenvector_right = scipy.linalg.eig(A, B)
    eigValInd=np.argsort(eigenvalue)
    eigValInd = eigValInd[::-1][0:d]
    eigenvector_right = eigenvector_right[:,eigValInd]
    return eigenvector_right

In [8]:
def get_U(X):
    r = np.linalg.matrix_rank(X)
    U, S, V = LA.svd(X, full_matrices=True)
    U = U[:,0:r]
    return U
def get_projections(X, L, L_bar, d=7):
    #######先进行SVD分解
    r = np.linalg.matrix_rank(np.matmul(X, X.T))
    U = get_U(X)
    X_tilde = np.matmul(U.T, X)
    
    A = np.matmul(np.matmul(X_tilde, L_bar), X_tilde.T)
    B = np.matmul(np.matmul(X_tilde, L), X_tilde.T)
    eigenvalue, eigenvector_right = scipy.linalg.eig(A, B)
    eigValInd=np.argsort(eigenvalue)
    eigValInd = eigValInd[::-1][0:d]
    eigenvector_right = eigenvector_right[:,eigValInd]
    return np.matmul(U, eigenvector_right)

In [100]:
Err_list = []
for i in xrange(1,51):
    #### 执行LDE降维过程
    train_face, test_face, train_label, test_label = load_ORL_data(randomsplit=i)
    mn = np.mean(train_face, keepdims=True, axis=1)
    train_face = train_face - mn
    test_face = test_face - mn
    neighbor_ind, neighbor_bar_ind, distance = get_neighbors_ind_and_distance(train_face, train_label, K=5, K_bar=5)
    Weight, Weight_bar = get_weight(neighbor_ind, neighbor_bar_ind, distance, t=5)
    Weight = np.maximum(Weight, Weight.T)
    Weight_bar = np.maximum(Weight_bar, Weight_bar.T)
    L, L_bar = get_Laplacian_matrix(Weight, Weight_bar)
    projections = get_projections(train_face, L, L_bar, d=39)
    ### 执行KNN分类过程
    ##train数据
    lowDim_TrainData = np.matmul(projections.T, train_face)
    ##test数据
    lowDim_TestData = np.matmul(projections.T, test_face)
    clf = neighbors.KNeighborsClassifier(algorithm='auto',n_neighbors=1, weights= 'distance')  
    clf.fit(lowDim_TrainData.T, train_label) 
    """测试准确率"""
    acc = np.float(sum(test_label==clf.predict(lowDim_TestData.T))) / len(test_label)
    Err_list.append(100.0 - acc*100)
print np.mean(Err_list), np.std(Err_list)

3.2625 1.5626999872


# Kernel LDE

In [30]:
def get_K(X1, X2, sigma=10):
    """
    X1: M * N1 matrix
    X2: M * N2 matrix
    """
    distance = EuclideanDistances(X1.T, X2.T)
    K = np.exp(-distance**2/(2*sigma**2))
    return K
def get_kernel_projections(K, L, L_bar, d=7):
#     U, S, V = LA.svd(K, full_matrices=True)
#     K = np.matmul(U.T, K)
    A = np.matmul(np.matmul(K, L_bar), K)
    B = np.matmul(np.matmul(K, L), K)
    eigenvalue, eigenvector_right = scipy.linalg.eig(A, B)
    eigValInd=np.argsort(eigenvalue)
    eigValInd = eigValInd[::-1][0:d]
    eigenvector_right = eigenvector_right[:,eigValInd]
    return eigenvector_right
#     return np.matmul(U, eigenvector_right)

In [103]:
Err_list = []
for i in xrange(1,51):
    #### 执行Kernel LDE降维过程
    sigma = 5.0
    train_face, test_face, train_label, test_label = load_ORL_data(randomsplit=i)
    neighbor_ind, neighbor_bar_ind, distance = get_neighbors_ind_and_distance(train_face, train_label, K=5, K_bar=5)
    Weight, Weight_bar = get_weight(neighbor_ind, neighbor_bar_ind, distance, t=5)
    Weight = np.maximum(Weight, Weight.T)
    Weight_bar = np.maximum(Weight_bar, Weight_bar.T)
    L, L_bar = get_Laplacian_matrix(Weight, Weight_bar)
    K_train = get_K(train_face, train_face, sigma=sigma)
    projections = get_kernel_projections(K_train, L, L_bar, d=39)
    K_test = get_K(train_face, test_face, sigma=sigma)
    ### 执行KNN分类过程
    ##train数据
    lowDim_TrainData = np.matmul(projections.T, K_train)
    ##test数据
    lowDim_TestData = np.matmul(projections.T, K_test)
    clf = neighbors.KNeighborsClassifier(algorithm='auto',n_neighbors=1, weights= 'distance')  
    clf.fit(lowDim_TrainData.T, train_label) 
    """测试准确率"""
    acc = np.float(sum(test_label==clf.predict(lowDim_TestData.T))) / len(test_label)
    Err_list.append(100.0 - acc*100)
print np.mean(Err_list), np.std(Err_list)

5.2125 2.42580527867


# 2DLDE

In [95]:
def FrobNorm(X): ###计算X元素与元素间 差 的 Frobenius 范数 
    """
    X: n1 * n2 * N tensor
    """
    n1, n2, N = X.shape
    FN = np.zeros((N,N))
    for i in xrange(N):
        for j in xrange(N):
            FN[i,j]= LA.norm(X[:,:,i]-X[:,:,j], ord='fro')
    return FN    

def get_2D_neighbors_ind_and_distance(X, label, K=3, K_bar=3):
    """
    Param:
        X: n1 * n2 * N tensor
        label: the label
        K: The number of neighbors of each point for G  ##  构建同类邻接图 G 的近邻数
        K_bar: The number of neighbors of each point for G_bar  ##  构建同类邻接图 G 的近邻数
    """    
    n1, n2, N = X.shape
    distance = FrobNorm(X)
    index = np.argsort(distance, axis=0)
    neighbor_ind = []
    neighbor_bar_ind = []
    for i in xrange(len(label)):
        index_i = index[:,i]  ### 第 i 个样本的全部邻居
        neighbor_ind_i = []    #### 与第i个样本是同类样本且是邻居
        neighbor_bar_ind_i = []  #### 与第i个样本不是同类样本且是邻居
        for ind in index_i[1:]: #### 从1开始是因为邻居不包括自己
            if (len(neighbor_ind_i) < K) and (label[i] == label[ind]):
                neighbor_ind_i.append(ind)
            if (len(neighbor_bar_ind_i) < K_bar) and (label[i] != label[ind]):
                neighbor_bar_ind_i.append(ind)
            if (len(neighbor_ind_i) == K) and (len(neighbor_bar_ind_i) == K_bar):
                neighbor_ind.append(neighbor_ind_i)
                neighbor_bar_ind.append(neighbor_bar_ind_i)
                break
    return neighbor_ind, neighbor_bar_ind, distance

In [119]:
Err_list = []
for i in xrange(1, 51):
    train_face, test_face, train_label, test_label = load_ORL_data(randomsplit=i)
    ### reshape 数据
    train_face_img = train_face.reshape((32, 32, -1))
    test_face_img = test_face.reshape((32, 32, -1))

    neighbor_ind, neighbor_bar_ind, distance = get_2D_neighbors_ind_and_distance(train_face_img, train_label, K=5, K_bar=5)
    Weight, Weight_bar = get_weight(neighbor_ind, neighbor_bar_ind, distance, t=5)

    l1 = 7
    l2 = 7
    n1 = train_face_img.shape[0]
    n2 = train_face_img.shape[1]
    N = train_face_img.shape[2]
    # L = np.eye(n1)[:,0:l1] #### 对L进行初始化
    L = np.ones((n1, l1))
    R = np.eye(n2)[:,0:l2]  #### 对R进行初始化

    itera = 10   ## 迭代次数
    R_change = []
    L_change = []
    for i in xrange(itera):
        #### flip-flop 的第1步， 给定 L，计算 R
        A = np.zeros((n2, n2))
        B = np.zeros((n2, n2))
        for i in xrange(N):
            for j in xrange(N):
                if (Weight[i,j] == 0) and (Weight_bar[i,j] == 0):
                    continue
                LL_T = np.matmul(L, L.T)
                p = np.matmul((train_face_img[:,:,i] - train_face_img[:,:,j]).T, LL_T)
                p = np.matmul(p, (train_face_img[:,:,i] - train_face_img[:,:,j]))
                A_tmp = Weight_bar[i,j] * p
                B_tmp = Weight[i,j] * p
                A = A + A_tmp
                B = B + B_tmp
        eigenvalue, eigenvector_right = scipy.linalg.eig(A, B)
        eigValInd=np.argsort(eigenvalue)
        eigValInd = eigValInd[::-1][0:l2]
        eigenvector_right = eigenvector_right[:,eigValInd]
        tol_R = LA.norm((eigenvector_right - R), ord='fro')   #### 用以判断 R 是否收敛
        R_change.append(tol_R)
        R = eigenvector_right

        #### flip-flop 的第2步， 给定 R，计算 L
        A = np.zeros((n1, n1))
        B = np.zeros((n1, n1))
        for i in xrange(N):
            for j in xrange(N):
                if (Weight[i,j] == 0) and (Weight_bar[i,j] == 0):
                    continue
                RR_T = np.matmul(R, R.T)
                p = np.matmul((train_face_img[:,:,i] - train_face_img[:,:,j]), RR_T)
                p = np.matmul(p, (train_face_img[:,:,i] - train_face_img[:,:,j]).T)
                A_tmp = Weight_bar[i,j] * p
                B_tmp = Weight[i,j] * p
                A = A + A_tmp
                B = B + B_tmp
        eigenvalue, eigenvector_right = scipy.linalg.eig(A, B)
        eigValInd=np.argsort(eigenvalue)
        eigValInd = eigValInd[::-1][0:l1]
        eigenvector_right = eigenvector_right[:,eigValInd]
        tol_L = LA.norm((eigenvector_right - L), ord='fro')   #### 用以判断 L 是否收敛
        L_change.append(tol_L)
        L = eigenvector_right
    
    ##train数据
    N_train = len(train_label)
    lowDim_TrainData = np.zeros((l1, l2, N_train))
    for i in xrange(N_train):
        lowDim_TrainData[:,:,i] = np.matmul(np.matmul(L.T, train_face_img[:,:,i]), R)
    ##test数据
    N_test = len(test_label)
    lowDim_TestData = np.zeros((l1, l2, N_test))
    for i in xrange(N_test):
        lowDim_TestData[:,:,i] = np.matmul(np.matmul(L.T, test_face_img[:,:,i]), R)

    ### 执行KNN分类过程
    ##train数据
    lowDim_TrainData = lowDim_TrainData.reshape((l1*l2, N_train))
    ##test数据
    lowDim_TestData = lowDim_TestData.reshape((l1*l2, N_test))
    clf = neighbors.KNeighborsClassifier(algorithm='auto',n_neighbors=1, weights= 'distance')  
    clf.fit(lowDim_TrainData.T, train_label) 
    """测试准确率"""
    acc = np.float(sum(test_label==clf.predict(lowDim_TestData.T))) / len(test_label)
    Err_list.append(100.0 - acc*100)
print np.mean(Err_list), np.std(Err_list)


4.025 1.3645054049
